In [25]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [26]:
# params < 20k ;  
# BN, Dropout, a Fully connected layer, GAP; 
# < 20 epochs;
#  > 99.4
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) # 30*30*1 | 3*3 | 28*28*32
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 30*30*32 | 3*3 | 28*28*64
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)  # 28*28*64 | 14*14*64
        self.drpout1 = nn.Dropout(p=0.25)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)  # 16*16*64 | 3*3 | 14*14*128
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  # 16*16*128 | 3*3 | 14*14*256
        self.bn2 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)  # 14*14*256 | 7*7*512
        self.drpout2 = nn.Dropout(p=0.25)
        self.gap1 =  nn.AvgPool2d(7) # 7*7*256 | 1*1*256
        self.fc1 = nn.Linear(256, 10)


    def forward(self, x):
        x = self.drpout1(self.pool1(self.bn1(F.relu(self.conv2(F.relu(self.conv1(x)))))))
        x = self.drpout2(self.pool2(self.bn2(F.relu(self.conv4(F.relu(self.conv3(x)))))))
        x = self.gap1(x)
        x = x.view(x.shape[0], -1)
        x = (self.fc1(x))
        return F.log_softmax(x)

In [27]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
       BatchNorm2d-3           [-1, 64, 28, 28]             128
         MaxPool2d-4           [-1, 64, 14, 14]               0
           Dropout-5           [-1, 64, 14, 14]               0
            Conv2d-6          [-1, 128, 14, 14]          73,856
            Conv2d-7          [-1, 256, 14, 14]         295,168
       BatchNorm2d-8          [-1, 256, 14, 14]             512
         MaxPool2d-9            [-1, 256, 7, 7]               0
          Dropout-10            [-1, 256, 7, 7]               0
        AvgPool2d-11            [-1, 256, 1, 1]               0
           Linear-12                   [-1, 10]           2,570
Total params: 391,050
Trainable params: 391,050
Non-trainable params: 0
-------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [28]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [31]:
#every epoch tracking
from tqdm import tqdm
def train(model, device, train_loader, optimizer):
    #pbar = tqdm(train_loader)
    for epoch in range(2):
      model.train()
      correct = 0
      loss = 0
    
      for data, target in train_loader:
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = F.nll_loss(output, target)
          loss.backward()
          optimizer.step()
          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()

        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

      print('\nTrain set: Accuracy: {}/{} ({:.4f}%)'.format(
        correct, len(train_loader.dataset),
          100. * correct / len(train_loader.dataset)))

      model.eval()
      test_loss = 0
      test_correct = 0
      with torch.no_grad():
          for data, target in test_loader:
              data, target = data.to(device), target.to(device)
              output = model(data)
              test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
              pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
              test_correct += pred.eq(target.view_as(pred)).sum().item()

      test_loss /= len(test_loader.dataset)

      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
          test_loss, test_correct, len(test_loader.dataset),
          100. * test_correct / len(test_loader.dataset)))
    
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [32]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train(model, device, train_loader, optimizer)
test(model, device, test_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Train set: Accuracy: 55633/60000 (92.7217%)

Test set: Average loss: 0.0865, Accuracy: 9772/10000 (97.7200%)


Train set: Accuracy: 58999/60000 (98.3317%)

Test set: Average loss: 0.0487, Accuracy: 9858/10000 (98.5800%)


Test set: Average loss: 0.0487, Accuracy: 9858/10000 (98.5800%)

